- 保监会 预警模型 2 训练 

# 基本设置

In [1]:
##load packages, needed
# encoding=utf-8

import jieba
import sys
import re
import time
import string
from sklearn import feature_extraction
from sklearn.pipeline import Pipeline,FeatureUnion

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest, chi2,mutual_info_classif,f_classif 

import xgboost as xgb
from xgboost import XGBClassifier
from sklearn import metrics

from sklearn.base import BaseEstimator, TransformerMixin
from collections import defaultdict

import pickle

import joblib
%matplotlib inline
import numpy as np
import pandas as pd
import pre
import os
from sklearn.model_selection import train_test_split
from sqlalchemy import create_engine
from pandas.io import sql

import warnings
warnings.filterwarnings('ignore')

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ADMINI~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.866 seconds.
Prefix dict has been built succesfully.


In [2]:
class StatsFeatures(BaseEstimator, TransformerMixin):
    
    def __init__(self):
        self.neg = set()
        f = open("corpus/neg_words.txt","r+", encoding='UTF-8')
        for content in f:
            self.neg.add(content)
        f.close()

    def fit(self, X, y=None):
        return self

    def getcnt(self,x):        
        return len(list(set(x)))

    def getnegcnt(self,x):
        negcnt = 0
        words = x.split()
        for w in words:
            if w in self.neg:
                negcnt = negcnt+1
        return negcnt
    
    def transform(self, X):
        data = []
        for x in X:
            if len(x) == 0:
                length  = 1
            else :
                length = len(x)
            data.append([len(x),self.getcnt(x),self.getcnt(x)/length,
                         self.getnegcnt(x),self.getnegcnt(x)/length])            
        return data

# 导入数据

In [4]:
# 预警数据
corpus_cor = []
label_cor = []

filename = 'data/corpus_pre_warn_0621.txt'
fid = open(filename, "r+", encoding='UTF-8')
for f in fid:
    corpus_cor.append(f)
    label_cor.append(1)
fid.close()
print(len(corpus_cor))
print(len(label_cor))

1628
1628


In [5]:
# 非预警数据
corpus_uncor = []
label_uncor = []

filename = 'data/corpus_pre_no_warn_0621.txt'
fid = open(filename, "r+", encoding='UTF-8')
for f in fid:
    corpus_uncor.append(f)
    label_uncor.append(0)
fid.close()
print(len(corpus_uncor))
print(len(label_uncor))

1923
1923


# 训练模型

## 分割训练集、测试集

In [6]:
# 未加入系统中噪音
corpus = corpus_cor + corpus_uncor
label = label_cor + label_uncor
X_train, X_test, y_train, y_test = train_test_split(corpus, label, test_size=0.3, random_state=42)
print('训练集：',len(X_train))
print('测试集：',len(X_test))

训练集： 2485
测试集： 1066


## train

In [8]:
pipeline = Pipeline([
    ('features', FeatureUnion([
        ('tf_idf', Pipeline([
            ('counts', CountVectorizer()),
            ('tf_idf', TfidfTransformer()),
            ('chi', SelectKBest(chi2, k=20000))
        ])),
        ('len_stats', StatsFeatures())
    ])),
    ('classifier', XGBClassifier(max_depth=7,objective='multi:softmax', num_class=2))
])

pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('tf_idf', Pipeline(memory=None,
     steps=[('counts', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0...tate=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1))])

In [9]:
print(pipeline.score(X_train, y_train))

0.9987927565392354


## test

In [10]:
y_pred_class = pipeline.predict(X_test)
print('accuracy_score: ', metrics.accuracy_score(y_test, y_pred_class)) # 指所有分类正确的百分比
print(metrics.classification_report(y_test, y_pred_class))
print('confusion_matrix: ')
print( metrics.confusion_matrix(y_test, y_pred_class))

accuracy_score:  0.924953095684803
             precision    recall  f1-score   support

          0       0.91      0.94      0.93       555
          1       0.94      0.90      0.92       511

avg / total       0.93      0.92      0.92      1066

confusion_matrix: 
[[524  31]
 [ 49 462]]


# 模型保存

In [11]:
from sklearn.externals import joblib
joblib.dump(pipeline, "model/circ_warning_0621.pkl.z")

['model/circ_warning_0621.pkl.z']